In [1]:
import csv
import os
from tqdm import tqdm
from pyfaidx import Fasta


In [2]:

file_path = "../extras/1-AFDBClusters-entryId_repId_taxId.tsv"

tracker ={}

all_members = []

# Open the TSV file in read mode with appropriate encoding
with open(file_path, 'r', encoding='utf-8') as tsvfile:
    # Use csv reader to read the TSV file, specifying the delimiter as '\t' (tab)
    tsvreader = csv.reader(tsvfile, delimiter='\t')
    
    # Iterate through each row in the TSV file
    for row in tsvreader:
        # Print each row
        memberID = row[0]
        repId = row[1]
        if repId not in tracker:
            tracker[repId] = {'size': 1, 'members': []}
        else:
            tracker[repId]['size'] += 1
        tracker[repId]['members'].append(memberID)
        all_members.append(memberID)

In [3]:
number_of_clusters = len(tracker)
print(f"Number of clusters: {number_of_clusters}")

Number of clusters: 2302908


In [4]:

def extract_id(header):
    return header.split('|')[1]

sequences = Fasta('../extras/uniprot_sprot.fasta', key_function=extract_id)

In [5]:

members_in_swissprot = {}
for member in all_members:
    if member not in sequences:
        continue
    sequence = sequences[member][:].seq
    seq_len = len(sequence)
    members_in_swissprot[member] = seq_len


In [6]:
print(f"Number of members in SwissProt: {len(members_in_swissprot)}")

Number of members in SwissProt: 95413


In [7]:
# delete the sequences that are not in SwissProt
for repId in tracker:
    members = tracker[repId]['members']
    members = [member for member in members if member in members_in_swissprot]
    tracker[repId]['member_sizes'] = [members_in_swissprot[member] for member in members]
    tracker[repId]['members'] = members
    tracker[repId]['size'] = len(members)

In [8]:
tracker = {repId: data for repId, data in tracker.items() if data['size'] >= 1}

In [9]:
print(f"Number of clusters with at least 1 members in SwissProt: {len(tracker)}")

Number of clusters with at least 1 members in SwissProt: 37783


In [18]:
# Make a list of the shortest member of each cluster
shortest_members = []
for repId in tracker:
    members = tracker[repId]['members']
    member_sizes = tracker[repId]['member_sizes']
    shortest_member = members[member_sizes.index(min(member_sizes))]
    if min(member_sizes) < 200:
        shortest_members.append(shortest_member)
print(f"Number of clusters with at least 1 member shorter than 200 residues: {len(shortest_members)}")

Number of clusters with at least 1 member shorter than 100 residues: 9616


In [19]:
counterrr = 0
c2=0
stds = []
for repId in tracker:
    members = tracker[repId]['members']
    member_sizes = tracker[repId]['member_sizes']
    std = max(member_sizes) - min(member_sizes)
    stds.append(std)
    shortest_member = members[member_sizes.index(min(member_sizes))]
    largest_member = members[member_sizes.index(max(member_sizes))]
    if largest_member == repId:
        c2 += 1
    if shortest_member == repId:
        counterrr += 1
print(max(stds))
print(min(stds))
print(c2)
print(f"Number of shortest members: {counterrr}")

4614
0
2367
Number of shortest members: 2406


In [21]:
import os
from tqdm import tqdm

# Delete all files that are not the shortest member of their cluster

# Replace 'directory_path' with the path of the directory you want to list files from
directory_path = 'D:/af50'

files = os.listdir(directory_path)
for file in tqdm(files, desc="Processing files"):
    pdb = file.split('-')[1]
    if pdb not in shortest_members:
        os.remove(os.path.join(directory_path, file))

Processing files:   0%|          | 0/26153 [00:00<?, ?it/s]

Processing files: 100%|██████████| 26153/26153 [00:34<00:00, 751.83it/s] 
